# Financial Analysis with Data Science
By Cyrill Adrian Wicaksono

This note book aims to extracting the crucial insights from financial markets.

# Essential Libraries



1.   **yfinance**: extract data from the financial market
2.  **Quanstats**: quantitative financial analysis tools
3. **PyPortofolioOpt**: optimization and asset allocation
4. **TA**: technical analysis library; provides multiple technical analysis indicators



In [4]:
#Installing yfinance
%pip install yfinance

#Installing Quantstats
%pip install quantstats

#Installing PyPortfolioOpt
%pip install pyportfolioopt

#Installing TA
%pip install ta


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: quantstats in c:\users\sonnie\appdata\local\programs\python\python311\lib\site-packages (0.0.62)




[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
%pip install scikit-learn


     ---------------------------------------- 10.6/10.6 MB 5.7 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
#Import libraries
import pandas as pd
import numpy as np
from scipy import stats

import quantstats as qs
import ta
import yfinance as yf

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

# Fundamental Analysis
This is the exploration process; using stocks for 4 tickers namely: BBRI, BBCA, BMRI, and BBNI


In [80]:
tickers = ["BBCA.JK", "BBNI.JK", "BMRI.JK", "BBRI.JK"]

## Average Daily Returns
A stock daily return is the percentual change in price over a single day. We calculate it between the difference between the stock's closing price on one day and its closing price the day before, dividing the result by closing of the day before, and multiplying it by 100.


For example:
<center>$((price On Monday - priceOnTuesday) \div 100) \times 100 = Daily Returns$</center>

In [20]:
returns_data = {}

for ticker in tickers:
    returns = qs.utils.download_returns(ticker)
    returns_data[ticker] = returns.loc['2010-07-01': '2024-02-10']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Calculate the average daily returns of each stock from the data


In [21]:

for ticker in tickers:
  print(f"Average Daily Returns {ticker} : {returns_data[ticker].mean()}")

Average Daily Returns BBCA.JK : 0.000746378538102235
Average Daily Returns BBNI.JK : 0.0007087290967559536
Average Daily Returns BMRI.JK : 0.0006728326821862299
Average Daily Returns BBRI.JK : 0.0007871482361318915


## Beta and Alpha
Beta and Alpha are the two key matrix used in finance to evaluate the performance of a stock relative to the overall market. Beta is a measure of a stock vollatility compared to the market. Alpha, on the other hand, is a measurement of a stock excess return relative to its expected performance based on its Beta.

In [22]:
ihsg = qs.utils.download_returns("^JKSE")
ihsg = ihsg.loc['2010-07-01': '2024-02-10']

returns_data["IHSG.JK"] = ihsg

[*********************100%%**********************]  1 of 1 completed


In [23]:
returns_no_index = {}

for ticker in tickers:
  returns_no_index[ticker] = returns_data[ticker].reset_index(drop=True)

returns_no_index['IHSG.JK'] = returns_data['IHSG.JK'].reset_index(drop=True)

In [29]:
def compute_alpha_beta(stock):
  X = returns_no_index["IHSG.JK"].values.reshape(-1, 1)
  y = stock.values.reshape(-1, 1)

  dataframe = {}
  if len(X) != len(y):
      min_len = min(len(X), len(y))
      X = X[:min_len]
      y = y[:min_len]

  model = LinearRegression()
  model.fit(X, y)

  beta = model.coef_[0]
  alpha = model.intercept_

  dataframe['beta'] = beta.round(3)
  dataframe['alpha'] = alpha.round(3)

  return dataframe


In [36]:
ab_df = pd.DataFrame(index=tickers, columns=['alpha', 'beta'])
for stock in returns_no_index.keys():
  ab_df.loc[stock] = compute_alpha_beta(returns_no_index[stock])

ab_df

,alpha,beta
BBCA.JK,[0.001],[0.411]
BBNI.JK,[0.001],[0.473]
BMRI.JK,[0.0],[0.535]
BBRI.JK,[0.001],[0.532]
IHSG.JK,[0.0],[1.0]


Beta values for all the stocks are greater than 1, meaning that they are more volatile than the benchmark and may offer higher returns, but also come with greater risks. On the other hand, the alpha values for all the stocks are small, close to zero, suggesting that there is little difference between the expected returns and the risk-adjusted returns

## Sharpe Ratio

Sharpe ratio measures of risk-adjusted return of investments. It is calculated by dividing the average excess return of investments over the standard deviation of returns. Here is the equation:

<div style="display: flex; justify-content: center;">
  <center><img src="https://latex.codecogs.com/png.image?\dpi{150}&space;\bg_white&space;\text{Sharpe&space;ratio}&space;=&space;\frac{R_p&space;-&space;R_f}{\sigma_p}" title="\text{Sharpe ratio} = \frac{R_p - R_f}{\sigma_p}" /></center>
</div>

where <b><i>R<sub>p</sub></i></b> is the average return of the investment, <i><b>R<sub>f</sub></b></i> is the risk-free rate of return, and <i><b>σ<sub>p</sub></b></i> is the standard deviation of the returns. The average excess return is the difference between the average return of the investment and the risk-free rate of return, typically represented by a government bond. The standard deviation is a measurement of the volatility of returns.</p>
<br>

A higher Sharpe ratio indicates that an investment provides higher returns for a given level of risk compared to other investments with a lower Sharpe ratio. In general, a Sharpe ratio greater than 1 is considered good, while a Sharpe ratio less than 1 is considered poor. A Sharpe ratio of 1 means that the investment's average return is equal to the risk-free rate of return.

In general, a Sharpe ratio under 1.0 is considered bad, equal to 1.0 is considered acceptable or good, 2.0 or higher is rated as very good, and 3.0 or higher is considered excellent.


In [43]:
for stock in returns_data.keys():
  print(f"Sharpe Ratio for {stock}: {qs.stats.sharpe(returns_data[stock]).round(2)}")

Sharpe Ratio for BBCA.JK: 0.75
Sharpe Ratio for BBNI.JK: 0.53
Sharpe Ratio for BMRI.JK: 0.52
Sharpe Ratio for BBRI.JK: 0.6
Sharpe Ratio for IHSG.JK: 0.51


None of the stocks have a Sharp ratio above 1, which may suggest that these investments' average returns are beneath the risk-free rate of return.

It's important to note that the Sharpe ratio is an annual metric and, since the beginning of 2022, the market, in general, has been bearish, with prices going down over the past year.

## Piotroski F-Score

The Piotroski F-score is a scoring system designed by Joseph Piotroski to assess the financial strength of a company based on its fundamental characteristics. The score ranges from 0 to 9, with a higher score indicating better financial health. Here's how the F-score is calculated based on the provided code:

1. **Net Income (NI) > 0 (Criteria 1):**
   - If the net income of the latest fiscal year is greater than zero, assign 1 point.

2. **Operating Cash Flow (OCF) > 0 (Criteria 2):**
   - If the operating cash flow of the latest fiscal year is greater than zero, assign 1 point.

3. **Return on Assets (ROA) > 0 (Criteria 3):**
   - Calculate the return on assets (net income divided by total assets).
   - If the ROA is greater than zero, assign 1 point.

4. **Cash Flow from Operations (OCF) > Net Income (NI) (Criteria 4):**
   - If the cash flow from operations of the latest fiscal year is greater than the net income, assign 1 point.

5. **Long-Term Debt (LT Debt) Decreased (Criteria 5):**
   - If the long-term debt decreased from the previous fiscal year to the latest fiscal year, assign 1 point.

6. **Current Ratio Increased (Criteria 6):**
   - If the current ratio (total current assets divided by total current liabilities) increased from the previous fiscal year to the latest fiscal year, assign 1 point.

7. **Shares Outstanding (SO) Decreased (Criteria 7):**
   - If the number of shares outstanding decreased from the previous fiscal year to the latest fiscal year, assign 1 point.

8. **Gross Margin (GM) Increased (Criteria 8):**
   - If the gross margin (total revenue minus cost of revenue) increased from the previous fiscal year to the latest fiscal year, assign 1 point.

Finally, the total Piotroski F-score is the sum of points assigned based on meeting each of the criteria.

In [77]:
# Fetch balance sheet data
balance_sheet = yf.Ticker("BBCA.JK").balance_sheet

# Fetch cash flow data
cash_flow = yf.Ticker("BBCA.JK").cashflow

# Fetch financial statement data
financials = yf.Ticker("BBCA.JK").financials

# Concatenate the data vertically
data = pd.concat([balance_sheet, cash_flow, financials]).transpose()
data.columns

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Net Tangible Assets', 'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest',
       ...
       'Depreciation And Amortization In Income Statement', 'Amortization',
       'Depreciation Income Statement', 'Selling General And Administration',
       'Selling And Marketing Expense', 'General And Administrative Expense',
       'Insurance And Claims', 'Rent And Landing Fees', 'Total Revenue',
       'Operating Revenue'],
      dtype='object', length=117)

In [81]:
def compute_piotroski_score(ticker):
    # Fetch balance sheet data
    balance_sheet = yf.Ticker(ticker).balance_sheet

    # Fetch cash flow data
    cash_flow = yf.Ticker(ticker).cashflow

    # Fetch financial statement data
    financials = yf.Ticker(ticker).financials

    # Concatenate the data vertically
    data = pd.concat([balance_sheet, cash_flow, financials]).transpose()
    dataframe = {}


    # Criteria 1: Net Income (NI) > 0
    try:
        if data['Net Income'][-1] > 0:
            dataframe["net_income"] = data['Net Income'][-1]
    except (KeyError, IndexError, TypeError) as e:
        dataframe["net_income"] = None
        print(f"Error in Criteria 1: {e}")

    # Criteria 2: Operating Cash Flow (OCF) > 0
    try:
        if data['Operating Cash Flow'][-1] > 0:
            dataframe["operating_cashflow"] = data['Operating Cash Flow'][-1]
    except (KeyError, IndexError, TypeError) as e:
        dataframe["operating_cashflow"] = None
        print(f"Error in Criteria 2: {e}")

    # Criteria 3: Return on Assets (ROA) > 0
    try:
        if data['Total Assets'][-1] > 0:
            roa = data['Net Income'][-1] / data['Total Assets'][-1]
            if roa > 0:
                dataframe['roa'] = roa
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        dataframe['roa'] = None
        print(f"Error in Criteria 3: {e}")

    # Criteria 4: Cash Flow from Operations (OCF) > Net Income (NI)
    try:
        if data['Operating Revenue'][-1] > data['Net Income'][-1]:
            dataframe['op_revenue_v_net_income_margin'] = data['Operating Revenue'][-1] - data['Net Income'][-1]
    except (KeyError, IndexError, TypeError) as e:
        dataframe['op_revenue_v_net_income_margin'] = None
        print(f"Error in Criteria 4: {e}")

    # Criteria 5: Long-Term Debt (LT Debt) Decreased
    try:
        if data['Long Term Debt And Capital Lease Obligation'][-1] < data['Long Term Debt And Capital Lease Obligation'][-2]:
                dataframe['lt_debt_decreased_percent'] = (data['Long Term Debt And Capital Lease Obligation'][-2] - data['Long Term Debt And Capital Lease Obligation'][-1]) / data['Long Term Debt And Capital Lease Obligation'][-2] * 100
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        dataframe['lt_debt_decreased_percent'] = None
        print(f"Error in Criteria 5: {e}")

    # Criteria 6: Current Ratio Increased
    try:
        if data['Total Liabilities Net Minority Interest' ][-1] > 0 and data['Total Assets'][-1] > data['Total Liabilities Net Minority Interest' ][-1]:
                dataframe['current_ratio'] = data['Total Liabilities Net Minority Interest' ][-1] / data['Total Assets'][-1]
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        dataframe['current_ratio'] = None
        print(f"Error in Criteria 6: {e}")

    # Criteria 7: Shares Outstanding (SO) Decreased
    try:
        if data['Diluted Average Shares'][-1] < data['Diluted Average Shares'][-2]:
                dataframe['percent_shares_outstanding'] = (data['Diluted Average Shares'][-2] - data['Diluted Average Shares'][-1]) / data['Diluted Average Shares'][-2] * 100
    except (KeyError, IndexError, TypeError, ZeroDivisionError) as e:
        dataframe['percent_shares_outstanding'] = None
        print(f"Error in Criteria 7: {e}")

    # Criteria 8: Gross Margin (GM) Increased
    try:
        if data['Total Revenue'][-1] > data['Operating Revenue'][-1]:
                dataframe['gross_margin'] = data['Total Revenue'][-1] - data['Operating Revenue'][-1]
    except (KeyError, IndexError, TypeError) as e:
        dataframe['gross_margin'] = None
        print(f"Error in Criteria 8: {e}")

    return dataframe


In [83]:

piotroski_df = pd.DataFrame(index=tickers, columns=['net_income', 'operating_cashflow', 'roa', 'op_revenue_v_net_income_margin', 'lt_debt_decreased_percent', 'current_ratio', 'percent_shares_outstanding', 'gross_margin'])

for ticker in tickers:
  result = compute_piotroski_score(ticker)
  dict = {'Test': 1, 'Test2': 2}
  dict.update(result)
  piotroski_df.loc[ticker] = result
  
piotroski_df.head()
dict
# piotroski_df.head()

{'Test': 1,
 'Test2': 2,
 'net_income': 18654753000000.0,
 'operating_cashflow': 29804584000000.0,
 'roa': 0.011586332858798556,
 'op_revenue_v_net_income_margin': 105487029000000.0,
 'current_ratio': 0.8574797707092315,
 'percent_shares_outstanding': 5.558729144722316}

## Revenue Growth

Calculate the percentage change in revenue over a specified period (e.g., year-over-year growth rate).

In [46]:
def compute_revenue_growth(ticker):
    revenue_growth = 0
    # Fetch historical revenue data
    revenue_data = yf.Ticker(ticker).financials.transpose()['Total Revenue']

    # Compute revenue growth percentage
    current_revenue = revenue_data[-1]
    previous_revenue = revenue_data[-2]
    revenue_growth = ((current_revenue - previous_revenue) / previous_revenue) * 100


    return revenue_growth

In [47]:
for ticker in tickers:
  growth_percentage = compute_revenue_growth(ticker)
  print(f"Revenue Growth {ticker}: {growth_percentage:.2f}%")

Revenue Growth BBCA.JK: -4.28%
Revenue Growth BBNI.JK: -4.78%
Revenue Growth BMRI.JK: -11.96%
Revenue Growth BBRI.JK: -16.01%


## Earning Per Share

Divide the company's net income by the number of outstanding shares.

In [48]:
def compute_eps(ticker):
  eps = 0
  net_income = yf.Ticker(ticker).financials.transpose()['Net Income']
  outstanding_shares = yf.Ticker(ticker).info['sharesOutstanding']

  eps = net_income[-1]/outstanding_shares

  return eps

In [49]:
for ticker in tickers:
  print(f"EPS scores for {ticker}: {compute_eps(ticker)}")

EPS scores for BBCA.JK: 220.0860563092296
EPS scores for BBNI.JK: 5668.729079429653
EPS scores for BMRI.JK: 179.99486939812365
EPS scores for BBRI.JK: 123.71264530913241


## Price-to-Earning

Divide the current market price per share by the earnings per share.

In [50]:
def compute_pe_ratio(ticker):
  pe_ratio = 0

  try:
    current_price = yf.Ticker(ticker).info['currentPrice']

    eps = yf.Ticker(ticker).financials.transpose()['Net Income'][-1] / yf.Ticker(ticker).info['sharesOutstanding']
    pe_ratio = current_price/eps
  except KeyError as e:
    print(f"KeyError: {e}")

  return pe_ratio

In [51]:
for ticker in tickers:
  print(f"PE Ratio for {ticker}: {compute_pe_ratio(ticker)}")

PE Ratio for BBCA.JK: 45.777547968982766
PE Ratio for BBNI.JK: 1.0407976668720276
PE Ratio for BMRI.JK: 40.2789258618478
PE Ratio for BBRI.JK: 48.903650753585424


## Debt-to-Equity

Divide total debt by shareholders' equity.

In [52]:
def compute_debt_to_equity_ratio(ticker):
  debt_to_equity_ratio = 0
  try:
      total_stockholder_equity = yf.Ticker(ticker).balance_sheet.transpose()['Stockholders Equity']
      total_debt = yf.Ticker(ticker).info['totalDebt']
      debt_to_equity_ratio = total_debt / total_stockholder_equity[-1]

  except KeyError as e:
    print(f"KeyError: {e}")

  return debt_to_equity_ratio

In [53]:
for ticker in tickers:
  print(f"Debt-to-Equity for {ticker}: {compute_debt_to_equity_ratio(ticker)}")

Debt-to-Equity for BBCA.JK: 0.03749632728183334
Debt-to-Equity for BBNI.JK: 0.48998715045413244
Debt-to-Equity for BMRI.JK: 1.018590890563945
Debt-to-Equity for BBRI.JK: 0.7651531417484786


## Free Cash Flow

Subtract capital expenditures from operating cash flow.

In [54]:
def compute_free_cashflow(ticker):
  financial = yf.Ticker(ticker).cashflow.transpose()

  op_cf = financial['Operating Cash Flow'][-1]
  capex = financial['Capital Expenditure'][-1]

  return  op_cf - capex


In [55]:
for ticker in tickers:
  print(f"Free Cashflow of {ticker}: {compute_free_cashflow(ticker)}")

Free Cashflow of BBCA.JK: 53652612000000.0
Free Cashflow of BBNI.JK: 75527364000000.0
Free Cashflow of BMRI.JK: 116599094000000.0
Free Cashflow of BBRI.JK: 34179650000000.0


## Return On Equity

Divide net income by shareholders' equity.

In [56]:
def compute_return_on_equity(ticker):
  total_stockholder_equity = yf.Ticker(ticker).balance_sheet.transpose()['Stockholders Equity'][-1]
  net_income = yf.Ticker(ticker).financials.transpose()['Net Income'][-1]

  roe = (net_income/total_stockholder_equity) * 100
  return roe


In [57]:
for ticker in tickers:
  print(f"Free Return On Equity of {ticker}: {compute_return_on_equity(ticker)}")

Free Return On Equity of BBCA.JK: 14.697534662742962
Free Return On Equity of BBNI.JK: 2.976974299162956
Free Return On Equity of BMRI.JK: 8.870868759396
Free Return On Equity of BBRI.JK: 8.22099314605118


## Dividend Yield

Divide the annual dividend per share by the current market price per share.

In [58]:
def compute_dividend_yield(ticker):
  dividends = yf.Ticker(ticker).dividends
  current_price = yf.Ticker(ticker).history(period='1d')['Close'][0]

  annual_dividend_per_share = dividends.sum()
  dividend_yield = (annual_dividend_per_share/current_price) * 100

  return dividend_yield


In [59]:
for ticker in tickers:
  print(f"Dividend Yield of {ticker}: {compute_dividend_yield(ticker)}")

Dividend Yield of BBCA.JK: 12.668982630272954
Dividend Yield of BBNI.JK: 25.57313977966102
Dividend Yield of BMRI.JK: 25.366445544827588
Dividend Yield of BBRI.JK: 27.39566069421488


# Pipelining

## Get Tickers


Get all the ticker that is publicly traded from the Jakarta Stock Exchange.

In [84]:
ticker_list = pd.read_csv("ticker_symbols.csv")

tickers = list(ticker_list['Ticker'])

In [85]:
len(tickers)

921

## Computation Function of the Financial Analysis

In [86]:
stock_analysis = {}

In [87]:
def compute_financial_analysis(ticker):
    result = {}

    try:
        # Compute Average Daily Returns
        returns = qs.utils.download_returns(ticker).loc['2020-07-01': '2024-02-10']
        result['Average Daily Returns'] = returns.mean()
    except Exception as e:
        result['Average Daily Returns'] = ""

    try:
        # Compute Beta-Alpha
        returns_no_index = returns.reset_index(drop=True)
        betaalpha = compute_alpha_beta(returns_no_index)
        result.update(betaalpha)
    except Exception as e:
        print(e)

    try:
        # Compute Sharpe Ratio
        result['Sharpe Ratio'] = qs.stats.sharpe(returns).round(2)
    except Exception as e:
        result['Sharpe Ratio'] = ""

    try:
        # Compute Piotroski F-score
        pio = compute_piotroski_score(ticker)
        result.update(pio)
        
    except Exception as e:
        print(e)

    try:
        # Compute Revenue Growth
        result['Revenue Growth'] = compute_revenue_growth(ticker)
    except Exception as e:
        result['Revenue Growth'] = ""

    try:
        # Compute Earning Per Share
        result['Earning per Share'] = compute_eps(ticker)
    except Exception as e:
        result['Earning per Share'] = ""

    try:
        # Compute Price to Earning
        result['Price to Earning'] = compute_pe_ratio(ticker)
    except Exception as e:
        result['Price to Earning'] = ""

    try:
        # Compute Debt to Equity
        result['Debt to Equity'] = compute_debt_to_equity_ratio(ticker)
    except Exception as e:
        result['Debt to Equity'] = ""

    try:
        # Compute Free Cashflow
        result['Free Cash Flow'] = compute_free_cashflow(ticker)
    except Exception as e:
        result['Free Cash Flow'] = ""

    try:
        # Compute Return on Equity
        result['Return on Equity'] = compute_return_on_equity(ticker)
    except Exception as e:
        result['Return on Equity'] = ""

    try:
        # Compute Dividend Yield
        result['Dividend Yield'] = compute_dividend_yield(ticker)
    except Exception as e:
        result['Dividend Yield'] = ""

    return result


Check for one of the ticker

In [88]:
test_ticker = tickers[0]
test_ticker

'AALI.JK'

In [89]:
print(f"{test_ticker}: {compute_financial_analysis(test_ticker)}")

[*********************100%%**********************]  1 of 1 completed


Error in Criteria 2: 'Operating Cash Flow'


AALI.JK: {'Average Daily Returns': -2.992250227229979e-05, 'beta': array([0.035]), 'alpha': array([-0.]), 'Sharpe Ratio': -0.03, 'net_income': 833090000000.0, 'operating_cashflow': None, 'roa': 0.0299875120724492, 'op_revenue_v_net_income_margin': 17973953000000.0, 'current_ratio': 0.30716554640793275, 'Revenue Growth': -22.67492030276398, 'Earning per Share': 432.8437198839821, 'Price to Earning': 15.941088395251413, 'Debt to Equity': 0.2135743776992211, 'Free Cash Flow': '', 'Return on Equity': 4.442555984422978, 'Dividend Yield': 129.82281157971016}


### Computes the Analysis for All the Tickers

In [90]:
for ticker in tickers:
  stock_analysis[ticker] = compute_financial_analysis(ticker)

[*********************100%%**********************]  1 of 1 completed


Error in Criteria 2: 'Operating Cash Flow'


[*********************100%%**********************]  1 of 1 completed


Error in Criteria 2: 'Operating Cash Flow'


[*********************100%%**********************]  1 of 1 completed


...

Error in Criteria 2: 'Operating Cash Flow'


[*********************100%%**********************]  1 of 1 completed


Error in Criteria 1: 'Net Income'
Error in Criteria 2: 'Operating Cash Flow'
Error in Criteria 3: 'Total Assets'
Error in Criteria 4: 'Operating Revenue'
Error in Criteria 5: 'Long Term Debt And Capital Lease Obligation'
Error in Criteria 6: 'Total Liabilities Net Minority Interest'
Error in Criteria 7: 'Diluted Average Shares'
Error in Criteria 8: 'Total Revenue'
KeyError: 'Net Income'
KeyError: 'Stockholders Equity'


[*********************100%%**********************]  1 of 1 completed


Input y contains NaN.
Error in Criteria 1: 'Net Income'
Error in Criteria 2: 'Operating Cash Flow'
Error in Criteria 3: 'Total Assets'
Error in Criteria 4: 'Operating Revenue'
Error in Criteria 5: 'Long Term Debt And Capital Lease Obligation'
Error in Criteria 6: 'Total Liabilities Net Minority Interest'
Error in Criteria 7: 'Diluted Average Shares'
Error in Criteria 8: 'Total Revenue'
KeyError: 'Net Income'
KeyError: 'Stockholders Equity'


In [91]:
stock_analysis

{'AALI.JK': {'Average Daily Returns': -2.992250227229979e-05,
  'beta': array([0.035]),
  'alpha': array([-0.]),
  'Sharpe Ratio': -0.03,
  'net_income': 833090000000.0,
  'operating_cashflow': None,
  'roa': 0.0299875120724492,
  'op_revenue_v_net_income_margin': 17973953000000.0,
  'current_ratio': 0.30716554640793275,
  'Revenue Growth': -22.67492030276398,
  'Earning per Share': 432.8437198839821,
  'Price to Earning': 15.941088395251413,
  'Debt to Equity': 0.2135743776992211,
  'Free Cash Flow': '',
  'Return on Equity': 4.442555984422978,
  'Dividend Yield': 129.82281157971016},
 'ABBA.JK': {'Average Daily Returns': 0.001393156250978238,
  'beta': array([-0.047]),
  'alpha': array([0.001]),
  'Sharpe Ratio': 0.45,
  'operating_cashflow': None,
  'op_revenue_v_net_income_margin': 203504123516.0,
  'Revenue Growth': -6.034068612093069,
  'Earning per Share': -11.406882837566759,
  'Price to Earning': -2.9806565460658887,
  'Debt to Equity': -20.456808251051843,
  'Free Cash Flow':

## Create Pandas DataFrame from the Calculation Result

In [92]:
analysis = pd.DataFrame.from_dict(stock_analysis, orient='index')
analysis.head()

,Average Daily Returns,beta,alpha,Sharpe Ratio,net_income,operating_cashflow,roa,op_revenue_v_net_income_margin,current_ratio,Revenue Growth,Earning per Share,Price to Earning,Debt to Equity,Free Cash Flow,Return on Equity,Dividend Yield,lt_debt_decreased_percent,percent_shares_outstanding,gross_margin
AALI.JK,-0.000030,[0.035],[-0.0],-0.03,8.330900e+11,NaN,0.029988,1.797395e+13,0.307166,-22.67492,432.84372,15.941088,0.213574,,4.442556,129.822812,NaN,NaN,NaN
ABBA.JK,0.001393,[-0.047],[0.001],0.45,NaN,NaN,NaN,2.035041e+11,NaN,-6.034069,-11.406883,-2.980657,-20.456808,,458.951936,0.000000,NaN,NaN,NaN
ABDA.JK,0.000081,[-0.006],[0.0],0.07,1.382038e+11,NaN,0.055777,7.689906e+11,0.440130,15.041351,3906.974495,1.279763,0.000000,,9.963071,20.786799,NaN,NaN,NaN
ABMM.JK,0.001542,[0.138],[0.001],0.73,NaN,NaN,NaN,6.420668e+08,0.804471,-40.656829,-0.012952,-294160.009784,5.803908,,-19.281866,19.259103,NaN,NaN,NaN
ACES.JK,-0.000341,[0.039],[-0.0],-0.21,7.331955e+11,NaN,0.101597,6.679571e+12,0.259554,13.28681,42.825836,20.548344,0.163593,,13.764517,25.743636,NaN,NaN,NaN


In [93]:
#Number of Columns

analysis.columns

Index(['Average Daily Returns', 'beta', 'alpha', 'Sharpe Ratio', 'net_income',
       'operating_cashflow', 'roa', 'op_revenue_v_net_income_margin',
       'current_ratio', 'Revenue Growth', 'Earning per Share',
       'Price to Earning', 'Debt to Equity', 'Free Cash Flow',
       'Return on Equity', 'Dividend Yield', 'lt_debt_decreased_percent',
       'percent_shares_outstanding', 'gross_margin'],
      dtype='object')

In [94]:
#Number of Null Values

analysis.isna().sum()

Average Daily Returns              10
beta                              235
alpha                             235
Sharpe Ratio                       50
net_income                        544
operating_cashflow                887
roa                               544
op_revenue_v_net_income_margin    329
current_ratio                     350
Revenue Growth                      0
Earning per Share                   0
Price to Earning                    0
Debt to Equity                      0
Free Cash Flow                      1
Return on Equity                    0
Dividend Yield                      0
lt_debt_decreased_percent         705
percent_shares_outstanding        800
gross_margin                      921
dtype: int64

In [96]:
#Number of Empty Strings

empty_counts = analysis.apply(lambda col: col[col == ""].count())
print(empty_counts)

Average Daily Returns               0
beta                                0
alpha                               0
Sharpe Ratio                        0
net_income                          0
operating_cashflow                  0
roa                                 0
op_revenue_v_net_income_margin      0
current_ratio                       0
Revenue Growth                    329
Earning per Share                 335
Price to Earning                    0
Debt to Equity                      0
Free Cash Flow                    867
Return on Equity                  319
Dividend Yield                      0
lt_debt_decreased_percent           0
percent_shares_outstanding          0
gross_margin                        0
dtype: int64


c:\Users\Sonnie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\ops\array_ops.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


## Write DataFrame to CSV

In [97]:
csv_filename = "fundamental_analysis_version.csv"
analysis.to_csv(csv_filename)